In [213]:
using DataFrames, DataFramesMeta, LinearAlgebra, Distributions, Optim, Revise, Plots

In [214]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule

In [215]:
# test imperfect overlap

In [216]:
# I = 100;              # Number of consumers
J = 3;                  # Number of firms
K = 1;                  # Product characteristics
nT = 10;                 # Number of markets
# β = [1, -1];            # Preferences
β = -1.;            # Preferences
varζ = 3;               # Variance of the random taste
rangeB = [100, 1000];       # consumers per market (e.g. census block)
# varX = 2;               # Variance of X
varξ = 1;               # Variance of xi
ζ_ = rand(Normal(0,1), 1, 100); # Draw shocks (less)
δ0 = zeros(1, size(ζ_, 2))


1×100 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [217]:
dist_thresh = 0.5
df = BLPmodule.build_dist_data(J, β, nT, rangeB, varζ, varξ, dist_thresh)
df[:, Cols("t", "j", r"q", r"B", r"agg")]

,t,j,q,q0,agg_qj,agg_q0,B,agg_B,agg_s,agg_s0
,Int64,Int64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Float64
1,1,1,140.308,181.797,1165.12,1489.55,616,4362,0.267106,0.341483
2,1,2,148.57,181.797,1423.78,1726.74,616,5124,0.277864,0.336991
3,1,3,145.326,181.797,2042.42,2488.69,616,7120,0.286857,0.349535
4,2,1,196.523,245.564,1165.12,1489.55,626,4362,0.267106,0.341483
5,2,3,183.913,245.564,2042.42,2488.69,626,7120,0.286857,0.349535
6,3,1,116.907,142.062,1165.12,1489.55,373,4362,0.267106,0.341483
7,3,3,114.031,142.062,2042.42,2488.69,373,7120,0.286857,0.349535
8,4,2,234.4,261.794,1423.78,1726.74,731,5124,0.277864,0.336991
9,4,3,234.805,261.794,2042.42,2488.69,731,7120,0.286857,0.349535


In [229]:
df_j = df[Not(findall(nonunique(df[!,[:j]]))), Cols("j", "dist", r"agg")]

,j,dist,agg_qj,agg_q0,agg_B,agg_s,agg_s0
,Int64,Float64,Float64,Float64,Int64,Float64,Float64
1,1,0.295122,1165.12,1489.55,4362,0.267106,0.341483
2,2,0.481277,1423.78,1726.74,5124,0.277864,0.336991
3,3,0.205558,2042.42,2488.69,7120,0.286857,0.349535


In [219]:
using BLPmodule
deltas = BLPmodule.loop(df_j.agg_qj);

iterations:2
deltas:[7.0605773124387206, 7.261067358009756, 7.621890274362473]


In [220]:
println(BLPmodule.simple_shares(deltas))
df_j.agg_qj

[1165.117608537323, 1423.7754045515073, 2042.4192158784454]


3-element Vector{Float64}:
 1165.1176085373243
 1423.7754045515067
 2042.419215878445

In [230]:
using BLPmodule
deltas = BLPmodule.loop(df_j.agg_s);

iterations:2
deltas:[-1.3201086343228527, -1.2806233050068698, -1.248772730043549]


In [232]:
println(BLPmodule.simple_shares(deltas))
df_j.agg_s

[0.2671062834794416, 0.2778640524105203, 0.2868566314436018]


3-element Vector{Float64}:
 0.2671062834794416
 0.2778640524105204
 0.2868566314436018

---

Stuff below are wrong (working with the original functions, trying to work with shares). 

In [221]:
aux_x = repeat([0.], J);
aux_t = repeat([1], J);

In [222]:
using BLPmodule
deltas = BLPmodule.compute_delta(df_j.agg_s, df_j.agg_s0, aux_x , ζ_, aux_t);
shares = BLPmodule.implied_shares(aux_x, ζ_, deltas, δ0)
println("computed shares:       ", shares)
println("true aggregate shares: ", df_j.agg_s)

final counter: 10001
deltas:[8.075277635145266, 8.114762964461251, 8.146613539424571]
computed shares:       [0.32107590933717406, 0.3340073177525479, 0.34481687435571723]
true aggregate shares: [0.2671062834794416, 0.2778640524105204, 0.2868566314436018]


In [223]:
# using quantities
using BLPmodule
deltas = BLPmodule.compute_delta(df_j.agg_qj, df_j.agg_q0, aux_x, ζ_, aux_t);
shares = BLPmodule.implied_shares(aux_x, ζ_, deltas, δ0)
println("computed shares:       ", shares)
println("true aggregate shares: ", df_j.agg_s)

final counter: 10001
deltas:[7.831237224455996, 8.031727270027027, 8.392550186379749]
computed shares:       [0.25154883915743215, 0.3073930456560951, 0.44095821663191237]
true aggregate shares: [0.2671062834794416, 0.2778640524105204, 0.2868566314436018]


In [224]:
# δ0 = zeros(1, size(ζ_, 2))
# for t in unique(T)
#     tind = df.t.==t
#     shares = BLPmodule.implied_shares(X_[tind,:], ζ_, deltas1, δ0)
#     println("computed shares: ", shares)
# end
# println(agg_shares)

In [225]:
# using BLPmodule
# Minimize GMM objective function
# varζ_ = optimize(x -> BLPmodule.GMM(q_j_, q0_, X_, Z_, ζ_, T, IV_, x[1])[1], [3.0], LBFGS()).minimizer[1];

In [226]:
# β_blp = BLPmodule.GMM(q_j_, q0_, X_, Z_, ζ_, T, IV_, varζ_)[2];

In [227]:
# print("Estimated BLP coefficients: $β_blp")

In [228]:
# # Compute logit estimate
# y = log.(df.s) - log.(df.s0)
# β_logit = inv(IV_' * X_) * (IV_' * y)
# print("Estimated logit coefficients: $β_logit")